![https://www.pngmart.com/files/5/Spaghetti-Transparent-Background.png](https://www.pngmart.com/files/5/Spaghetti-Transparent-Background.png)

In [654]:
import pandas as pd
from math import *
import numpy as np
from scipy import interpolate

# Импорт компосовской херни
import pythoncom
from win32com.client import Dispatch, gencache

import LDefin2D
import MiscellaneousHelpers as MH



In [655]:
#  Подключим константы API Компас
kompas6_constants = gencache.EnsureModule("{75C9F5D0-B5B8-4526-8681-9903C567D2ED}", 0, 1, 0).constants
kompas6_constants_3d = gencache.EnsureModule("{2CAF168C-7961-4B90-9DA2-701419BEEFE3}", 0, 1, 0).constants

#  Подключим описание интерфейсов API5
kompas6_api5_module = gencache.EnsureModule("{0422828C-F174-495E-AC5D-D31014DBBE87}", 0, 1, 0)
kompas_object = kompas6_api5_module.KompasObject(
    Dispatch("Kompas.Application.5")._oleobj_.QueryInterface(kompas6_api5_module.KompasObject.CLSID,
                                                             pythoncom.IID_IDispatch))
MH.iKompasObject = kompas_object

#  Подключим описание интерфейсов API7
kompas_api7_module = gencache.EnsureModule("{69AC2981-37C0-4379-84FD-5DD2F3C0A520}", 0, 1, 0)
application = kompas_api7_module.IApplication(
    Dispatch("Kompas.Application.7")._oleobj_.QueryInterface(kompas_api7_module.IApplication.CLSID,
                                                             pythoncom.IID_IDispatch))
MH.iApplication = application

documents = application.Documents
#  Получим активный документ
kompas_document = application.ActiveDocument
kompas_document_2d = kompas_api7_module.IKompasDocument2D(kompas_document)
iDocument2D = kompas_object.ActiveDocument2D()

In [656]:
table_79 = pd.read_excel('Геодезия.xlsm', sheet_name='pandasPlan79')

table_79

,№№,X,Y,H
0,ПЗ41,1461.310000,2370.570000,132.240000
1,111,1626.253069,2371.784960,136.588469
2,112,1594.848907,2496.474264,132.671694
3,113,1542.897439,2664.448824,133.288756
4,ПЗ42,1336.480000,2587.050000,131.590000


In [657]:
table_10 = pd.read_excel('Геодезия.xlsm', sheet_name='pandasPlan10')

table_10

,№№,"Отсчет по дальномеру kl, м",Отсчет по гориз. Кругу,"Отметки реечн. точек Hр.т., м"
0,ПЗ41 - ПЗ42,NaN,NaN,NaN
1,1,88.0,12.033333,129.550000
2,2,111.0,78.183333,128.540000
3,3,155.0,136.400000,128.950000
4,4,132.0,144.916667,128.910000
5,5,141.0,195.450000,132.320000
6,111 - ПЗ41,NaN,NaN,NaN
7,6,94.0,125.416667,133.638469
8,7,131.0,181.350000,133.928469
9,8,128.0,231.250000,132.538469


In [658]:
def add_layer(id: int, active: bool = True, name: str = None):
    obj = iDocument2D.ksLayer(id)
    iLayerParam = kompas6_api5_module.ksLayerParam(kompas_object.GetParamStruct(kompas6_constants.ko_LayerParam))
    iLayerParam.Init()
    if name:
        iLayerParam.name = name
    iLayerParam.state = int(active)
    iDocument2D.ksSetObjParam(obj, iLayerParam, LDefin2D.ALLPARAM)


def add_view(name: str, scale: float = 1, x=0, y=0):
    iViewParam = kompas6_api5_module.ksViewParam(kompas_object.GetParamStruct(kompas6_constants.ko_ViewParam))
    iViewParam.Init()
    iViewParam.angle = 0
    iViewParam.color = 0
    iViewParam.name = name
    iViewParam.scale_ = scale
    iViewParam.state = 3
    iViewParam.x = x
    iViewParam.y = y
    iDocument2D.ksCreateSheetView(iViewParam, 0)


def add_text(text: str, x: float, y: float, angle: float = 0, font_size: float = 2.5):
    iParagraphParam = kompas6_api5_module.ksParagraphParam(
        kompas_object.GetParamStruct(kompas6_constants.ko_ParagraphParam))
    iParagraphParam.Init()
    iParagraphParam.x = x
    iParagraphParam.y = y
    iParagraphParam.ang = angle
    iParagraphParam.height = 3.55
    iParagraphParam.width = 4
    iParagraphParam.hFormat = 0
    iParagraphParam.vFormat = 0
    iParagraphParam.style = 1
    iDocument2D.ksParagraph(iParagraphParam)

    iTextLineParam = kompas6_api5_module.ksTextLineParam(
        kompas_object.GetParamStruct(kompas6_constants.ko_TextLineParam))
    iTextLineParam.Init()
    iTextLineParam.style = 1
    iTextItemArray = kompas_object.GetDynamicArray(LDefin2D.TEXT_ITEM_ARR)
    iTextItemParam = kompas6_api5_module.ksTextItemParam(
        kompas_object.GetParamStruct(kompas6_constants.ko_TextItemParam))
    iTextItemParam.Init()
    iTextItemParam.iSNumb = 0
    iTextItemParam.s = text
    iTextItemParam.type = 0
    iTextItemFont = kompas6_api5_module.ksTextItemFont(iTextItemParam.GetItemFont())
    iTextItemFont.Init()
    iTextItemFont.bitVector = 4096
    iTextItemFont.color = 0
    iTextItemFont.fontName = "GOST type A"
    iTextItemFont.height = font_size
    iTextItemFont.ksu = 1
    iTextItemArray.ksAddArrayItem(-1, iTextItemParam)
    iTextLineParam.SetTextItemArr(iTextItemArray)

    iDocument2D.ksTextLine(iTextLineParam)
    obj = iDocument2D.ksEndObj()



In [659]:
ONE_TO_SCALE = 2000

add_view("План", 1/ONE_TO_SCALE)
add_layer(1,True,"Основные Точки")

for i, row in table_79.iterrows():
    # X и Y перевёрнуты
    obj = iDocument2D.ksCircle(row["Y"] * 1000, row["X"] * 1000, 1000, 1)
    add_text(str(row["№№"]) + " (" + str("{:1.2f}".format(row["H"])).replace(".", ",") + ")", row["Y"] * 1000 - 15000, row["X"] * 1000 - 7000)


In [660]:
# https://stackoverflow.com/questions/7586063/how-to-calculate-the-angle-between-a-line-and-the-horizontal-axis
def angle_trunc(a):
    while a < 0.0:
        a += pi * 2
    return a

def getAngleBetweenPoints(x_orig, y_orig, x_landmark, y_landmark):
    deltaY = y_landmark - y_orig
    deltaX = x_landmark - x_orig
    return angle_trunc(atan2(deltaY, deltaX))

In [661]:
table_10

,№№,"Отсчет по дальномеру kl, м",Отсчет по гориз. Кругу,"Отметки реечн. точек Hр.т., м"
0,ПЗ41 - ПЗ42,NaN,NaN,NaN
1,1,88.0,12.033333,129.550000
2,2,111.0,78.183333,128.540000
3,3,155.0,136.400000,128.950000
4,4,132.0,144.916667,128.910000
5,5,141.0,195.450000,132.320000
6,111 - ПЗ41,NaN,NaN,NaN
7,6,94.0,125.416667,133.638469
8,7,131.0,181.350000,133.928469
9,8,128.0,231.250000,132.538469


In [662]:
add_layer(2,True,"Побочные Точки")

extra_points = {}

for i, row in table_10.iterrows():
    if " - " in str(row["№№"]):
        origin_label, dest_label = row["№№"].split(" - ")

        if origin_label.isdigit():
            origin_label = int(origin_label)

        if dest_label.isdigit():
            dest_label = int(dest_label)

        origin = (table_79.loc[table_79['№№'] == origin_label]["Y"].values[0], table_79.loc[table_79['№№'] == origin_label]["X"].values[0])

        _dest = (table_79.loc[table_79['№№'] == dest_label]["Y"].values[0], table_79.loc[table_79['№№'] == dest_label]["X"].values[0])

        angle = np.rad2deg(getAngleBetweenPoints( origin[0], origin[1], _dest[0], _dest[1]))
        _i_am_shit_at_coding = list(origin)
        _i_am_shit_at_coding.append(table_79.loc[table_79['№№'] == origin_label]["H"].values[0])
        extra_points.update({str(origin_label): _i_am_shit_at_coding})
        continue
    else:
        if not origin:
            continue
    _d = row["Отсчет по дальномеру kl, м"]
    _alpha = row["Отсчет по гориз. Кругу"]
    xx = origin[0] + (_d * cos(np.radians(angle - _alpha)))
    yy = origin[1] + (_d * sin(np.radians(angle - _alpha)))
    obj = iDocument2D.ksLineSeg(origin[0] * 1000, origin[1] * 1000, xx * 1000,yy * 1000, 6)

    add_text(str(row["№№"]) + " (" + str("{:1.3f}".format(row["Отметки реечн. точек Hр.т., м"])).replace(".", ",") + ")", xx * 1000 - 10000, yy * 1000 - 8000)


    extra_points.update({str(row["№№"]): [xx, yy, row["Отметки реечн. точек Hр.т., м"]]})



In [663]:
interpolated_points = {}

In [664]:
# CON_P41 = "1,2,4,5,21".split(",")
# CON_1 = ["21"]
# CON_5 =["4"]

CON_111 = "5,6,7,8,9,112,ПЗ41".split(",")

center_point = np.array(extra_points["111"])
for point in CON_111:
    point = np.array(extra_points[point])

    if center_point[2] > point[2]:
        _coordinate_dif = center_point[:2] - point[:2]
        xia = [center_point[2], point[2]]
        hight_dif = range(int(ceil(point[2])), int(round(center_point[2])))
    else:
        _coordinate_dif = point[:2] - center_point[:2]
        xia = [point[2], center_point[2]]
        hight_dif = range(int(ceil(center_point[2])), int(round(point[2])))

    length = round(np.sqrt(np.sum(np.power(_coordinate_dif,2))))
    yia = [0, length]

    _alpha = np.rad2deg(getAngleBetweenPoints( center_point[0], center_point[1], point[0], point[1]))

    x = [0, length]
    f = interpolate.interp1d(xia, yia)

    for i in hight_dif:
        _d = f(i)
        xx = center_point[0] * 1000 + (_d * 1000 * cos(np.radians(_alpha)))
        yy = center_point[1] * 1000 + (_d * 1000 * sin(np.radians(_alpha)))
        obj = iDocument2D.ksPoint(xx  ,yy , 0)

        if i in interpolated_points:
            interpolated_points[i].append((xx, yy))
        else:
            interpolated_points.update({i: [(xx, yy)]})


In [665]:
x = [extra_points[p][0] for p in extra_points]
y = [extra_points[p][1] for p in extra_points]
centroid = (sum(x) / len(extra_points), sum(y) / len(extra_points))
obj = iDocument2D.ksPoint(*(cord * 1000 for cord in centroid), 0)

current_view = kompas_document_2d.ViewsAndLayersManager.Views.ActiveView
current_view.X = 594/2 - centroid[0] / 2
current_view.Y = 420/2 - centroid[1] / 2
current_view.Update()


True

In [666]:
for height in interpolated_points:

    cent=(sum([p[0] for p in interpolated_points[height]])/len(interpolated_points[height]),sum([p[1] for p in interpolated_points[height]])/len(interpolated_points[height]))
    interpolated_points[height].sort( key=lambda p: atan2(p[1]-cent[1],p[0]-cent[0]))

    obj = iDocument2D.ksBezier(True, 1)
    for point in interpolated_points[height]:
        dot = iDocument2D.ksPoint(*point, 0)
    iDocument2D.ksEndObj()